# 🏠 Analyse Exploratoire - Données Immobilières Paris

Ce notebook présente une analyse exploratoire complète des données immobilières parisiennes.

## Objectifs
1. Comprendre la distribution des prix
2. Identifier les facteurs clés influençant les prix
3. Analyser les tendances par arrondissement
4. Préparer les données pour la modélisation

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Warnings
import warnings
warnings.filterwarnings('ignore')

print('✅ Librairies importées')

## 1. Chargement des données

In [ ]:
# Charger les données
df = pd.read_csv('../data/immobilier_paris.csv')

print(f"📊 Dataset: {df.shape[0]} lignes, {df.shape[1]} colonnes")
print(f"\n📋 Aperçu:")
df.head()

In [ ]:
# Informations générales
df.info()

In [ ]:
# Statistiques descriptives
df.describe()

## 2. Analyse univariée - Prix

In [ ]:
# Distribution des prix
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Histogramme
axes[0].hist(df['prix'], bins=50, color='steelblue', alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Prix (€)', fontsize=12)
axes[0].set_ylabel('Fréquence', fontsize=12)
axes[0].set_title('Distribution des Prix', fontsize=14, fontweight='bold')
axes[0].axvline(df['prix'].median(), color='red', linestyle='--', label=f"Médiane: {df['prix'].median():,.0f}€")
axes[0].axvline(df['prix'].mean(), color='green', linestyle='--', label=f"Moyenne: {df['prix'].mean():,.0f}€")
axes[0].legend()

# Boxplot
axes[1].boxplot(df['prix'], vert=False)
axes[1].set_xlabel('Prix (€)', fontsize=12)
axes[1].set_title('Boxplot des Prix', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"Prix moyen: {df['prix'].mean():,.0f} €")
print(f"Prix médian: {df['prix'].median():,.0f} €")
print(f"Écart-type: {df['prix'].std():,.0f} €")

In [ ]:
# Prix au m²
df['prix_m2'] = df['prix'] / df['surface_m2']

plt.figure(figsize=(12, 5))
plt.hist(df['prix_m2'], bins=50, color='coral', alpha=0.7, edgecolor='black')
plt.xlabel('Prix au m² (€)', fontsize=12)
plt.ylabel('Fréquence', fontsize=12)
plt.title('Distribution du Prix au m²', fontsize=14, fontweight='bold')
plt.axvline(df['prix_m2'].median(), color='red', linestyle='--', label=f"Médiane: {df['prix_m2'].median():,.0f}€/m²")
plt.legend()
plt.show()

print(f"Prix moyen au m²: {df['prix_m2'].mean():,.0f} €")
print(f"Prix médian au m²: {df['prix_m2'].median():,.0f} €")

## 3. Analyse par arrondissement

In [ ]:
# Prix moyen par arrondissement
prix_par_arrond = df.groupby('arrondissement').agg({
    'prix': 'mean',
    'prix_m2': 'mean',
    'surface_m2': 'mean'
}).round(0)

prix_par_arrond.columns = ['Prix moyen', 'Prix moyen/m²', 'Surface moyenne']
prix_par_arrond = prix_par_arrond.sort_values('Prix moyen/m²', ascending=False)

print("Top 5 arrondissements les plus chers au m²:")
prix_par_arrond.head()

In [ ]:
# Visualisation interactive avec Plotly
fig = px.bar(
    prix_par_arrond.reset_index(),
    x='arrondissement',
    y='Prix moyen/m²',
    title='Prix moyen au m² par arrondissement',
    labels={'Prix moyen/m²': 'Prix au m² (€)', 'arrondissement': 'Arrondissement'},
    color='Prix moyen/m²',
    color_continuous_scale='Viridis'
)

fig.update_layout(height=500, showlegend=False)
fig.show()

In [ ]:
# Nombre d'annonces par arrondissement
annonces_par_arrond = df['arrondissement'].value_counts().sort_index()

plt.figure(figsize=(14, 5))
plt.bar(annonces_par_arrond.index, annonces_par_arrond.values, color='skyblue', alpha=0.7)
plt.xlabel('Arrondissement', fontsize=12)
plt.ylabel('Nombre d\'annonces', fontsize=12)
plt.title('Nombre d\'annonces par arrondissement', fontsize=14, fontweight='bold')
plt.xticks(range(1, 21))
plt.grid(axis='y', alpha=0.3)
plt.show()

## 4. Corrélations et relations entre variables

In [ ]:
# Matrice de corrélation
correlation_cols = ['surface_m2', 'nb_pieces', 'nb_chambres', 'arrondissement', 
                   'etage', 'annee_construction', 'balcon', 'parking', 
                   'distance_metro_m', 'prix']

plt.figure(figsize=(12, 10))
corr = df[correlation_cols].corr()
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={'shrink': 0.8})
plt.title('Matrice de corrélation', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nTop 5 corrélations avec le prix:")
print(corr['prix'].sort_values(ascending=False)[1:6])

In [ ]:
# Relation Surface vs Prix
fig = px.scatter(
    df.sample(1000),  # Échantillon pour la lisibilité
    x='surface_m2',
    y='prix',
    color='arrondissement',
    title='Relation entre Surface et Prix',
    labels={'surface_m2': 'Surface (m²)', 'prix': 'Prix (€)'},
    opacity=0.6,
    trendline='ols'
)

fig.update_layout(height=600)
fig.show()

## 5. Impact des équipements

In [ ]:
# Comparaison prix avec/sans équipements
equipements = ['balcon', 'terrasse', 'parking', 'cave', 'ascenseur', 'renovation_recente']

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for idx, equip in enumerate(equipements):
    data_avec = df[df[equip] == 1]['prix_m2']
    data_sans = df[df[equip] == 0]['prix_m2']
    
    axes[idx].boxplot([data_sans, data_avec], labels=['Sans', 'Avec'])
    axes[idx].set_title(f'{equip.replace("_", " ").title()}', fontweight='bold')
    axes[idx].set_ylabel('Prix au m² (€)')
    
    # Calculer la différence
    diff_pct = ((data_avec.mean() - data_sans.mean()) / data_sans.mean() * 100)
    axes[idx].text(0.5, 0.95, f'Impact: {diff_pct:+.1f}%',
                 transform=axes[idx].transAxes,
                 ha='center', va='top',
                 bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

## 6. Analyse temporelle

In [ ]:
# Prix par période de construction
df['periode'] = pd.cut(df['annee_construction'], 
                       bins=[1800, 1900, 1950, 1980, 2000, 2024],
                       labels=['Avant 1900', '1900-1950', '1950-1980', '1980-2000', 'Après 2000'])

prix_par_periode = df.groupby('periode')['prix_m2'].mean().sort_values()

plt.figure(figsize=(12, 6))
prix_par_periode.plot(kind='bar', color='teal', alpha=0.7)
plt.xlabel('Période de construction', fontsize=12)
plt.ylabel('Prix moyen au m² (€)', fontsize=12)
plt.title('Prix moyen au m² par période de construction', fontsize=14, fontweight='bold')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Insights clés

### Résumé de l'analyse exploratoire:

1. **Distribution des prix**:
   - Prix médian: ~740,000€
   - Grande variabilité selon l'arrondissement
   
2. **Facteurs les plus impactants**:
   - Surface (corrélation: +0.85)
   - Arrondissement (jusqu'à 100% de différence)
   - Nombre de pièces (corrélation: +0.72)
   
3. **Impact des équipements**:
   - Terrasse: +12%
   - Parking: +10%
   - Rénovation récente: +12%
   - Balcon: +8%
   
4. **Tendances géographiques**:
   - Les arrondissements 7, 8, 16 sont les plus chers
   - Les arrondissements périphériques (18-20) sont plus abordables
   
### Prochaines étapes:
- Feature engineering pour améliorer la prédiction
- Entraînement de modèles de Machine Learning
- Optimisation des hyperparamètres